In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('usecase_5').getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

In [3]:
df_911 = spark.read.format('csv').options(header=True, inferSchema=True).load('/FileStore/tables/911.csv')

In [4]:
df_911.count()

Out[5]: 99492

In [5]:
df_911.show(5)

+----------+-----------+--------------------+-----+--------------------+-------------------+-----------------+--------------------+---+
 lat| lng| desc| zip| title| timeStamp| twp| addr| e|
+----------+-----------+--------------------+-----+--------------------+-------------------+-----------------+--------------------+---+
40.2978759|-75.5812935|REINDEER CT & DEA...|19525|EMS: BACK PAINS/I...|2015-12-10 17:40:00| NEW HANOVER|REINDEER CT & DEA...| 1|
40.2580614|-75.2646799|BRIAR PATH & WHIT...|19446|EMS: DIABETIC EME...|2015-12-10 17:40:00|HATFIELD TOWNSHIP|BRIAR PATH & WHIT...| 1|
40.1211818|-75.3519752|HAWS AVE; NORRIST...|19401| Fire: GAS-ODOR/LEAK|2015-12-10 17:40:00| NORRISTOWN| HAWS AVE| 1|
 40.116153| -75.343513|AIRY ST & SWEDE S...|19401|EMS: CARDIAC EMER...|2015-12-10 17:40:01| NORRISTOWN| AIRY ST & SWEDE ST| 1|
 40.251492|-75.6033497|CHERRYWOOD CT & D...| null| EMS: DIZZINESS|2015-12-10 17:40:01| LOWER POTTSGROVE|CHERRYWOOD CT & D...| 1|
+----------+-----------+--------------------+-----+--------------------+-------------------+-----------------+--------------------+---+
only showing top 5 rows

In [6]:
df_911.printSchema()

root
-- lat: double (nullable = true)
-- lng: double (nullable = true)
-- desc: string (nullable = true)
-- zip: integer (nullable = true)
-- title: string (nullable = true)
-- timeStamp: timestamp (nullable = true)
-- twp: string (nullable = true)
-- addr: string (nullable = true)
-- e: integer (nullable = true)

In [7]:
df_911.createOrReplaceTempView('table_911')

In [8]:
zip_df = spark.read.format('csv').options(header=True, inferSchema=True).load('/FileStore/tables/zipcode.csv')

In [9]:
zip_df.count()

Out[10]: 43191

In [10]:
zip_df.show(5)

+---+----------+-----+---------+----------+--------+---+
zip| city|state| latitude| longitude|timezone|dst|
+---+----------+-----+---------+----------+--------+---+
210|Portsmouth| NH|43.005895|-71.013202| -5| 1|
211|Portsmouth| NH|43.005895|-71.013202| -5| 1|
212|Portsmouth| NH|43.005895|-71.013202| -5| 1|
213|Portsmouth| NH|43.005895|-71.013202| -5| 1|
214|Portsmouth| NH|43.005895|-71.013202| -5| 1|
+---+----------+-----+---------+----------+--------+---+
only showing top 5 rows

In [11]:
zip_df.printSchema()

root
-- zip: integer (nullable = true)
-- city: string (nullable = true)
-- state: string (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- timezone: integer (nullable = true)
-- dst: integer (nullable = true)

In [12]:
zip_df.createOrReplaceTempView('table_zip')

In [13]:
final_df = spark.sql("SELECT t1.*, t2.city, t2.state FROM table_911 t1 JOIN table_zip t2 ON t1.zip=t2.zip")

In [14]:
final_df.columns

Out[15]: ['lat',
 'lng',
 'desc',
 'zip',
 'title',
 'timeStamp',
 'twp',
 'addr',
 'e',
 'city',
 'state']

In [15]:
final_df.select('title','city','state').show(5)

+--------------------+-------------+-----+
 title| city|state|
+--------------------+-------------+-----+
EMS: BACK PAINS/I...|Gilbertsville| PA|
EMS: DIABETIC EME...| Lansdale| PA|
 Fire: GAS-ODOR/LEAK| Norristown| PA|
EMS: CARDIAC EMER...| Norristown| PA|
 EMS: HEAD INJURY| Lansdale| PA|
+--------------------+-------------+-----+
only showing top 5 rows

In [16]:
final_df.createOrReplaceTempView('data')

In [17]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import desc

In [18]:
def trimTitle(str1):
    str1 = str1.strip()
    rstr = ''
    if (str1 != None):
        rstr = str1.split(':')[0]
    return rstr

In [19]:
spark.udf.register('trimTitle',trimTitle,StringType())

Out[20]: <function __main__.trimTitle(str1)>

In [20]:
spark.sql("SELECT count(*) as count, state, trimTitle(title) as title from data group by state,trimTitle(title)").orderBy(desc('count')).show(5)

+-----+-----+-------+
count|state| title|
+-----+-----+-------+
44326| PA| EMS|
29297| PA|Traffic|
13012| PA| Fire|
 1| TX| EMS|
 1| AL|Traffic|
+-----+-----+-------+

In [21]:
spark.sql("SELECT count(*) as count, city, trimTitle(title) as title from data group by city,trimTitle(title)").orderBy(desc('count')).show(5)

+-----+----------+-------+
count| city| title|
+-----+----------+-------+
 7409|Norristown| EMS|
 4237| Pottstown| EMS|
 2919|Norristown|Traffic|
 2665| Lansdale| EMS|
 1822| Pottstown|Traffic|
+-----+----------+-------+
only showing top 5 rows